In [1]:
import os
from dotenv import find_dotenv, load_dotenv
from openai import OpenAI
import openai
import psycopg2

In [2]:
load_dotenv(find_dotenv())

True

In [3]:
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
# Set up your OpenAI API key
# openai.api_key = OPENAI_API_KEY
client = OpenAI()
# Choose a model
model = "text-embedding-ada-002"
print(OPENAI_API_KEY[:12])

sk-proj-SYay


In [4]:
print("Connecting to PostgreSQL...")
conn = psycopg2.connect(
    database="postgres",
    user="postgres",
    password="postgres",
    host="host.docker.internal",
)

cur = conn.cursor()

print("Successfully connected to PostgreSQL.")

Connecting to PostgreSQL...
Successfully connected to PostgreSQL.


In [5]:
# Converting the prompt to the pgvector embedding
def get_embedding(prompt):
    response = openai.embeddings.create(input=prompt, model="text-embedding-ada-002")

    embedding = response.data[0].embedding

    # Converting the embedding to the pgvector and returning it
    return "[" + ",".join(map(str, embedding)) + "]"

In [6]:
try:

    query = "Potentiodynamic polarization"
    query = " better time"
    query = "What are AI agents"
    query = "What is the LUCI framework"
    # Perform a cosine similarity search
    sql = f"""
       SELECT file_id, element_text FROM tbl_doc_elements WHERE element_text @@ websearch_to_tsquery('english','{query}') LIMIT 5;
        """
    cur.execute(sql)

    print("Query:", query)

    print("Most similar sentences:")

    for row in cur.fetchall():

        print(row)

except Exception as e:

    print("Error executing query", str(e))

Query: What is the LUCI framework
Most similar sentences:
('1d00e23c-84aa-4b43-933b-b46cd60e5c4f', 'Abstract - In this paper, we introduce the LUCI framework, the first AI-powered medical agent, to empower developers to build advanced medical applications and products. Luci is the first AI-based medical agent designed to automate a wide range of healthcare processes. LUCI will also be very suitable for medical scribing, as its voice agent simplifies the creation of clinical notes. The chain-of- thought mechanism introduced in this framework allows the user to pass multiple prompts in a go for the model to serve different functionalities. As an example, one prompt can be used for searching medical data, and another for writing. This is an innovative way that will revolutionize the field. Moreover, LUCI applies RxNorms to ensure consistency in medical nomenclature, which makes functionality even more effective.')
('1d00e23c-84aa-4b43-933b-b46cd60e5c4f', 'In the new era of generative AI t

In [7]:
cur.close()
conn.close()